## 목포
- 머신러닝 라이브러리인 sklearn의 사용법 숙지
- 머신러인 모델링을 직접해보기
- 비만도데이터를 기반으로 학습하고 예측하는 모델 만들기

#### 1. 모델정의


In [ ]:
from sklearn import neighbors
# KNN알고리즘으로 학습하는 모델 사용 
from sklearn.neighbors import KNeighborsClassifier #KNN 클래스
bmi_model = KNeighborsClassifier(n_neighbors=5) # 가장 가까운 이웃 수 

#### 2. 모델학습
- 데이터를 오픈해서 탐새갛는 과정
- 학습을 위해 문제, 답을 추출하는 과정

In [ ]:
# 현재 작업 경로
!pwd

/content


In [ ]:
# 현재 폴더 내부 상황
!ls

drive  sample_data


In [ ]:
# 작업 경로 변경하기
%cd ./drive/MyDrive/Colab\ Notebooks/22.04.11\ 머신러닝

/content/drive/MyDrive/Colab Notebooks/22.04.11 머신러닝


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/22.04.11 머신러닝


In [ ]:
# 데이터 불러오기
# 1. 판다스 import
import pandas as pd
# 2. csv 파일 로딩
bmi_data = pd.read_csv('./data/bmi_lbs.csv')
# 3. 위쪽 5개 살펴보기
bmi_data.head()

,Label,Gender,Height,Weight(lbs)
0,Obesity,Male,174,211.6416
1,Normal,Male,189,191.8002
2,Obesity,Female,185,242.5060
3,Overweight,Female,195,229.2784
4,Overweight,Male,149,134.4806


In [ ]:
# 비만도 컬럼의 종류를 확인해보자.
bmi_data['Label'].unique()

array(['Obesity', 'Normal', 'Overweight', 'Extreme Obesity', 'Weak',
       'Extremely Weak'], dtype=object)

In [ ]:
# 비만도 컬럼의 종류를 확인해보자.
bmi_data['Label'].value_counts()

Extreme Obesity    198
Obesity            130
Normal              69
Overweight          68
Weak                22
Extremely Weak      13
Name: Label, dtype: int64

In [ ]:
from pandas.core.frame import DataFrame
# 비만도 컬럼의 종류를 확인해보자.
df = DataFrame(bmi_data)
df.drop_duplicates(['Label'])

,Label,Gender,Height,Weight(lbs)
0,Obesity,Male,174,211.6416
1,Normal,Male,189,191.8002
3,Overweight,Female,195,229.2784
6,Extreme Obesity,Male,147,202.8232
27,Weak,Male,187,136.6852
60,Extremely Weak,Female,191,119.0484


In [ ]:
# 몸무게를 파운드 -> kg 단위로 변경
bmi_data['Weight(kg)'] = bmi_data['Weight(lbs)'] * 0.453592
bmi_data.head()

,Label,Gender,Height,Weight(lbs),Weight(kg)
0,Obesity,Male,174,211.6416,95.998937
1,Normal,Male,189,191.8002,86.999036
2,Obesity,Female,185,242.5060,109.998782
3,Overweight,Female,195,229.2784,103.998848
4,Overweight,Male,149,134.4806,60.999324


In [ ]:
# 기술통계 확인 
bmi_data.describe()

,Height,Weight(lbs),Weight(kg)
count,500.000000,500.000000,500.000000
mean,169.944000,233.687600,105.998826
std,16.375261,71.390696,32.382249
min,140.000000,110.230000,49.999446
25%,156.000000,176.368000,79.999114
50%,170.500000,233.687600,105.998826
75%,184.000000,299.825600,135.998494
max,199.000000,352.736000,159.998228


In [ ]:
# 문제와 답을 분리
X = bmi_data[['Height','Weight(kg)']]
y = bmi_data['Label']

In [ ]:
X.shape, y.shape

((500, 2), (500,))

In [ ]:
bmi_model.fit(X,y)

KNeighborsClassifier()

#### 3. 모델예측

In [ ]:
# 샘플데이터 추출
X_sample = bmi_data.iloc[[10,76,111,342,485],[2,4]]

In [ ]:
X_sample

,Height,Weight(kg)
10,195,80.999103
76,165,103.998848
111,146,156.998261
342,154,91.998981
485,173,110.998770


In [ ]:
y_sample = bmi_data.iloc[[10,76,11,342,485],0]

In [ ]:
y_sample

10      Normal
76     Obesity
11     Obesity
342    Obesity
485    Obesity
Name: Label, dtype: object

In [ ]:
pre = bmi_model.predict(X_sample)
pre

array(['Normal', 'Obesity', 'Extreme Obesity', 'Obesity', 'Obesity'],
      dtype=object)

In [ ]:
bmi_data.head()

,Label,Gender,Height,Weight(lbs),Weight(kg)
0,Obesity,Male,174,211.6416,95.998937
1,Normal,Male,189,191.8002,86.999036
2,Obesity,Female,185,242.5060,109.998782
3,Overweight,Female,195,229.2784,103.998848
4,Overweight,Male,149,134.4806,60.999324


#### 4. 모델평가

In [ ]:
# 모델평가 함수 활용하기
from sklearn.metrics import accuracy_score #정확도(전체중에서 정확히 맞춘 비율)

In [ ]:
score = accuracy_score(y_sample,pre)
score

0.8

### 문제점
- 500명의 비만도 데이터를 학습하고 그중에 5명을 추출해서 평가를 진행하는 방법은 옳지않다.
- 이미 모델이 500명에 대해서 학습했기 때문에 상대적으로 추출한 5명을 잘 맞출 확률이 높다.
- 그래서 머신러닝에서는 훈련용데이터와 평가용데이터를 사전에 구분해서 활용한다.
- 일반적으로 비율은 7:3으로 활용한다.

In [ ]:
from sklearn.model_selection import train_test_split #훈련용, 평가용 데이터 분리 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=219) # 호출할때마다 동일한 학습 데이터세트를 생성하기위해 주어지는 난수값
X_train.head()

,Height,Weight(kg)
175,185,118.998682
397,169,135.998494
121,187,121.998649
35,189,131.998538
125,158,95.998937


In [ ]:
X_train.shape, y_train.shape

((350, 2), (350,))

In [ ]:
X_test.shape, y_test.shape

((150, 2), (150,))

In [ ]:
# 모델정의 > 모델학습 > 모델예측 > 모델 평가
bmi_model2 = KNeighborsClassifier()
bmi_model2.fit(X_train,y_train)
pre2 = bmi_model2.predict(X_test)
score2 = accuracy_score(y_test,pre2)
score2

0.9066666666666666

In [ ]:
# 모델활용
bmi_model2.predict([[175,65],[188,65]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


array(['Normal', 'Weak'], dtype=object)

### 다양한 평가지표
- precision(정밀도) : 모델이 해당 클래스로 예측한것에 대한 정확도 
- recall(재현율) : 실제 해당 클래스에 대해서 맞춘 비율 
- f1-score
- ROC curve


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,pre2))

                 precision    recall  f1-score   support

Extreme Obesity       0.94      0.94      0.94        53
 Extremely Weak       1.00      0.75      0.86         4
         Normal       0.89      0.85      0.87        20
        Obesity       0.89      0.91      0.90        44
     Overweight       0.91      0.88      0.89        24
           Weak       0.71      1.00      0.83         5

       accuracy                           0.91       150
      macro avg       0.89      0.89      0.88       150
   weighted avg       0.91      0.91      0.91       150

